In [26]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

from data_sources.get import get_indicators

In [27]:
df = get_indicators(
    ['SP.POP.TOTL', 'FP.CPI.TOTL.ZG', 'SP.DYN.LE00.IN', 'NE.EXP.GNFS.ZS', 'NY.GDP.PCAP.KD', 'SL.UEM.TOTL.ZS', 'NV.AGR.TOTL.ZS', 'EG.ELC.ACCS.ZS', 'AG.LND.FRST.ZS', 'SH.DYN.MORT', 'NY.GDP.TOTL.RT.ZS', 'SP.DYN.TFRT.IN', 'EN.URB.LCTY.UR.ZS', 'IP.JRN.ARTC.SC', 'MS.MIL.XPND.GD.ZS']
)

print(df['Indicator Name'].unique())

['Population, total' 'Inflation, consumer prices (annual %)'
 'Life expectancy at birth, total (years)'
 'Exports of goods and services (% of GDP)'
 'GDP per capita (constant 2015 US$)'
 'Unemployment, total (% of total labor force) (modeled ILO estimate)'
 'Agriculture, forestry, and fishing, value added (% of GDP)'
 'Access to electricity (% of population)' 'Forest area (% of land area)'
 'Mortality rate, under-5 (per 1,000 live births)'
 'Total natural resources rents (% of GDP)'
 'Fertility rate, total (births per woman)'
 'Population in the largest city (% of urban population)'
 'Scientific and technical journal articles'
 'Military expenditure (% of GDP)']


In [28]:
df = df.pivot_table(values='Value',
                    index='Year',
                    columns=['Indicator Name', 'Country Name'])

In [29]:
df_nans = df.loc[2000:2019].isnull().sum().sort_values(ascending=False).reset_index()

In [30]:
countries_with_nans = df_nans[df_nans[0] > 2]['Country Name'].unique()
countries_with_nans

array(['Pacific island small states', 'Somalia', 'Turkmenistan',
       'Caribbean small states', 'Afghanistan', 'Tuvalu', 'Dominica',
       'New Caledonia', 'Isle of Man', 'Liechtenstein', 'Cayman Islands',
       'French Polynesia', 'San Marino', 'Faroe Islands',
       'Marshall Islands', 'St. Kitts and Nevis', 'Palau', 'Nauru',
       'Eritrea', 'Sint Maarten (Dutch part)', 'Andorra', 'Uzbekistan',
       'Papua New Guinea', 'Suriname', 'Maldives', 'Guyana', 'Barbados',
       'Djibouti', 'Libya', 'Solomon Islands', 'Haiti',
       'Syrian Arab Republic', 'Venezuela, RB', 'South Sudan',
       'Channel Islands', 'Armenia', 'Timor-Leste', 'Equatorial Guinea',
       'Turks and Caicos Islands', 'Yemen, Rep.', 'Small states',
       'Ethiopia', 'Curacao', 'Other small states', 'Qatar', 'Zimbabwe',
       'Aruba', 'Turkey', 'Bermuda', 'Cuba', 'Myanmar', 'Malaysia',
       'Lebanon', 'Sudan', "Korea, Dem. People's Rep.",
       'Central African Republic', 'Algeria', 'Kosovo',
       'U

In [31]:
df_cleared = df.loc[2000:2019].stack()
df_cleared.drop(index=countries_with_nans, level=1, inplace=True)

In [32]:
df_cleared.bfill(inplace=True)
df_cleared.ffill(inplace=True)

In [33]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_cleared.values)
df_scaled = pd.DataFrame(data=data_scaled, index=df_cleared.index, columns=df_cleared.columns)
df_scaled.head()

Indicator Name                    Access to electricity (% of population)  \
Year Country Name                                                           
2000 Africa Eastern and Southern                                -2.510399   
     Africa Western and Central                                 -1.938829   
     Albania                                                     0.624136   
     Angola                                                      0.524342   
     Antigua and Barbuda                                         0.524342   

Indicator Name                    Agriculture, forestry, and fishing, value added (% of GDP)  \
Year Country Name                                                                              
2000 Africa Eastern and Southern                                           0.342165            
     Africa Western and Central                                            1.323454            
     Albania                                                               1.698047            
     Angola                                                               -0.443243            
     Antigua and Barbuda                                                  -0.905311            

Indicator Name                    Exports of goods and services (% of GDP)  \
Year Country Name                                                            
2000 Africa Eastern and Southern                                 -0.512874   
     Africa Western and Central                                  -0.139560   
     Albania                                                     -0.727230   
     Angola                                                       1.809889   
     Antigua and Barbuda                                          2.135033   

Indicator Name                    Fertility rate, total (births per woman)  \
Year Country Name                                                            
2000 Africa Eastern and Southern                                  2.184429   
     Africa Western and Central                                   2.525287   
     Albania                                                     -0.360271   
     Angola                                                       2.978119   
     Antigua and Barbuda                                         -0.328243   

Indicator Name                    Forest area (% of land area)  \
Year Country Name                                                
2000 Africa Eastern and Southern                      0.125685   
     Africa Western and Central                      -0.526203   
     Albania                                         -0.185064   
     Angola                                           1.617897   
     Antigua and Barbuda                             -0.532414   

Indicator Name                    GDP per capita (constant 2015 US$)  \
Year Country Name                                                      
2000 Africa Eastern and Southern                           -0.662428   
     Africa Western and Central                            -0.664618   
     Albania                                               -0.627431   
     Angola                                                -0.603661   
     Antigua and Barbuda                                   -0.040773   

Indicator Name                    Inflation, consumer prices (annual %)  \
Year Country Name                                                         
2000 Africa Eastern and Southern                               0.411844   
     Africa Western and Central                               -0.259095   
     Albania                                                  -0.533270   
     Angola                                                   35.380077   
     Antigua and Barbuda                                      -0.453500   

Indicator Name                    Life expectancy at birth, total (years)  \
Year Country Name                                                           
2000 Africa Eastern and Southern             

In [34]:
df_cleared.to_csv('../data/dataset.csv')
df_scaled.to_csv('../data/dataset_normalized.csv')